# ITCR - Sede Interuniversitaria de Alajuela

## Introducción a Spark

### Profesora: María Mora

## Objetivos

### General

Entender y aplicar técnicas de análisis de grandes cantidades de datos para la resolución de problemas concretos a través de tecnologías de manipulación, extracción y sintetización estadística. El ejemplo está basado en [1].

Objetivos específicos

- Aplicar bibliotecas para la transformación de datos a gran escala para poder sintetizar el conocimiento para futuros análisis.

- Introduccir al estudiante en el uso de un framework para trabajo con big data como es Spark. 


# Datos

Se utilizarán dos conjuntos de datos 

1) Tienda en línea. Datos generados automáticamente. A manera de ejemplo, se simulará la generación de datos para una tienda en línea. Se aplicarán ciertas operaciones básicas sobre los datos para mostrar lo que este tipo de plataformas ofrecen. 

2) New York Times Best Sellers. 
Conjunto de datos bajado de Kaggle, los datos recopilados incluyen el título del libro, el autor, la fecha de la lista de best sellers, la fecha de publicación de la lista, la descripción del libro, el editor, el número de semanas en la lista, el rango (esta semana y la semana pasada) y precio.

Fuente: https://www.kaggle.com/cmenca/new-york-times-hardcover-fiction-best-sellers


# Contenidos


- Introducción al procesamiento a gran escala por medio de los siguientes ejemplos:
- Ejemplo 1. Tienda en línea.  
  - Ejemplo de procesamiento de data frames
  - Procesamiento de atributos
- Ejemplo 2. NYT Best Sellers.
  - Procesamiento de data frames


## Dependencias

In [1]:
# Bibliotecas requeridas para el ejercicio

import pandas as pd
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import date, timedelta, datetime
import time

from pyspark.sql import SparkSession, Row, dataframe

# generar valores enteros random
from random import seed
from random import randint
from random import random

import findspark
SPARK_PATH = '/opt/spark'
findspark.init(SPARK_PATH)

## 1. Introducción a procesamiento a gran escala


Inicialización de la sesión de Spark, punto de entrada a la programación con el API de Datasets, DataFrame y funcionalidad SQL.

In [3]:
#spark = SparkSession.builder.appName("PysparkEj1")\
#     .config ("spark.sql.shuffle.partitions", "50")\
#     .config("spark.driver.maxResultSize","5g")\
#     .config ("spark.sql.execution.arrow.enabled", "true")\
#     .getOrCreate()

POSTGRESQL_URL = "jdbc:postgresql://localhost/"
POSTGRESQL_USER = "postgres"
POSTGRESQL_PASSWORD = "big_data"

def create_spark_session():
    """
    This function builds a Spark Session
    return the main entry of a Spark DataFrame
    """
    spark = SparkSession \
      .builder \
      .appName("Basic JDBC pipeline") \
      .config("spark.driver.extraClassPath", "postgresql-42.1.4.jar") \
      .config("spark.executor.extraClassPath", "postgresql-42.1.4.jar") \
      .getOrCreate()
    return spark

spark = create_spark_session()

In [7]:
# Funciones utilitarias

def get_random_date(year):
    # Genera fechas random en el año solicitado por el usuario.
    # Params: 
    #   year: año en el que se generará la fecha
    
    # try to get a date
    try:
        #retorna datetime.datetime.strptime('{} {}'.format(randint(1, 366), year), '%j %Y')
        return datetime.strptime('{} {}'.format(randint(1, 366), year), '%j %Y')

    # si el valor se encuentra en el rango de los años bisiestos, inténtelo de nuevo
    except ValueError:
        get_random_date(year)  

## Ejemplo 1. Tienda en línea

In [8]:
# Generación de datos 

# Parametros

maxCustomerId = 100     # Max id para clientes
minAnio = 1900          # Min year para generar transacciones
maxAnio = 2019          # Max year para generar transacciones
maxValue = 1000         # Max value para amount
minValue = 50           # Min value para amount
amountTransactions = 100 # Numero de transacciones generadas automáticamente. 

def createTransactions(n) : 
   #Genera n transacciones. 
   #Parámetros: 
   #    n la cantidad de registros a ser generados.
   
   df = pd.DataFrame(columns=['customer_id', 'purchased_at', 'amount'])
  
   seed(1)
   for x in range(n):
      df.loc[x, 'customer_id'] = randint(0, maxCustomerId)
      df.loc[x, 'purchased_at'] = get_random_date(randint(minAnio, maxAnio))
      df.loc[x, 'amount'] = int(minValue + (random() * (maxValue - minValue)))
       
   return df

#Crea un Pandas DataFrame
dfPandas = createTransactions(amountTransactions)
print(dfPandas)

# Crea un Spark Data Frame
df = spark.createDataFrame(dfPandas)

   customer_id         purchased_at amount
0           17  1972-02-02 00:00:00    292
1           63  1997-08-19 00:00:00    498
2           48  2000-04-17 00:00:00    139
3            3  2014-07-19 00:00:00    461
4           97  1998-01-02 00:00:00    711
5           34  1992-04-27 00:00:00    611
6           13  2015-06-12 00:00:00     79
7            3  1983-10-05 00:00:00     58
8           48  1987-04-21 00:00:00    970
9           92  1903-09-28 00:00:00    260
10          56  1963-10-11 00:00:00    271
11          29  1986-04-23 00:00:00    772
12          37  2018-01-12 00:00:00    445
13          71  2018-11-25 00:00:00    145
14          80  1992-05-31 00:00:00    164
15          42  2014-12-31 00:00:00    525
16          54  1964-12-09 00:00:00    230
17          36  1975-09-13 00:00:00    853
18          64  1950-10-29 00:00:00    860
19          61  1931-07-26 00:00:00    443
20          22  1946-10-08 00:00:00    888
21          99  1986-07-11 00:00:00    132
22         

### Transfromaciones básicas
Spark provee funciones para realizar transformaciones a los datos. Por ejemplo, transformar fechas a una representación con formato específico.

In [9]:
# Nueva columna con los datos en el formato deseado
formatted_df = df.withColumn("date_string", date_format(col("purchased_at"), 'dd/MM/yyyy'))
formatted_df.show()

# Estructura del dataframe
formatted_df.printSchema()

+-----------+-------------------+------+-----------+
|customer_id|       purchased_at|amount|date_string|
+-----------+-------------------+------+-----------+
|         17|1972-02-02 00:00:00|   292| 02/02/1972|
|         63|1997-08-19 00:00:00|   498| 19/08/1997|
|         48|2000-04-17 00:00:00|   139| 17/04/2000|
|          3|2014-07-19 00:00:00|   461| 19/07/2014|
|         97|1998-01-02 00:00:00|   711| 02/01/1998|
|         34|1992-04-27 00:00:00|   611| 27/04/1992|
|         13|2015-06-12 00:00:00|    79| 12/06/2015|
|          3|1983-10-05 00:00:00|    58| 05/10/1983|
|         48|1987-04-21 00:00:00|   970| 21/04/1987|
|         92|1903-09-28 00:00:00|   260| 28/09/1903|
|         56|1963-10-11 00:00:00|   271| 11/10/1963|
|         29|1986-04-23 00:00:00|   772| 23/04/1986|
|         37|2018-01-12 00:00:00|   445| 12/01/2018|
|         71|2018-11-25 00:00:00|   145| 25/11/2018|
|         80|1992-05-31 00:00:00|   164| 31/05/1992|
|         42|2014-12-31 00:00:00|   525| 31/12

En el caso que se necesite crear una función que no es parte de la biblioteca estándar en Spark, es posible definir funciones creadas por el usuario (User Defined Functions o udf). La noción básica de una udf en Spark es un lambda acompañado por el tipo de dato retornado. Lo anterior es estrictamente necesario en lenguajes con un sistema de tipos débil.

Ejemplo:


In [10]:
string_to_date = \
    udf(lambda text_date: datetime.strptime(text_date, '%d/%m/%Y'),
        DateType())

typed_df = formatted_df.withColumn("date", string_to_date(formatted_df.date_string))
typed_df.show()
typed_df.printSchema()

+-----------+-------------------+------+-----------+----------+
|customer_id|       purchased_at|amount|date_string|      date|
+-----------+-------------------+------+-----------+----------+
|         17|1972-02-02 00:00:00|   292| 02/02/1972|1972-02-02|
|         63|1997-08-19 00:00:00|   498| 19/08/1997|1997-08-19|
|         48|2000-04-17 00:00:00|   139| 17/04/2000|2000-04-17|
|          3|2014-07-19 00:00:00|   461| 19/07/2014|2014-07-19|
|         97|1998-01-02 00:00:00|   711| 02/01/1998|1998-01-02|
|         34|1992-04-27 00:00:00|   611| 27/04/1992|1992-04-27|
|         13|2015-06-12 00:00:00|    79| 12/06/2015|2015-06-12|
|          3|1983-10-05 00:00:00|    58| 05/10/1983|1983-10-05|
|         48|1987-04-21 00:00:00|   970| 21/04/1987|1987-04-21|
|         92|1903-09-28 00:00:00|   260| 28/09/1903|1903-09-28|
|         56|1963-10-11 00:00:00|   271| 11/10/1963|1963-10-11|
|         29|1986-04-23 00:00:00|   772| 23/04/1986|1986-04-23|
|         37|2018-01-12 00:00:00|   445|

Spark SQL. Para sumar los datos podemos utilizar agrupamiento igual que en SQL groupBy. 

In [12]:
# Sumar todas las compras por cliente y día:

sum_df = typed_df.groupBy("customer_id", "date").sum()
sum_df.show()


+-----------+----------+----------------+-----------+
|customer_id|      date|sum(customer_id)|sum(amount)|
+-----------+----------+----------------+-----------+
|         75|1953-03-24|              75|        159|
|         83|1940-01-21|              83|         75|
|         57|1901-05-24|              57|        287|
|         13|1955-07-13|              13|        816|
|         17|1972-02-02|              17|        292|
|         34|1992-04-27|              34|        611|
|          2|1928-01-10|               2|        427|
|         52|1909-07-14|              52|        872|
|         67|1921-12-03|              67|        309|
|         34|1946-06-01|              34|        586|
|         86|1909-02-12|              86|        874|
|         50|1982-03-29|              50|        210|
|         71|2012-01-20|              71|        611|
|         40|1957-07-20|              40|        347|
|          4|1992-03-23|               4|        473|
|         77|1999-09-08|    

Spark sumará todas las columnas que no se encuentren especificadas en la operación groupBy. Algunos resultados no tienen interpretación últil. Por ejemplo, sumar la columna customer_id no da ningún valor agregado. 

Finalmente, es posible dar a las columnas un nombre más apropiado para los usuarios de los datos.

In [13]:
stats_df = \
    sum_df.select(
        col('customer_id'),
        col('date'),
        col('sum(amount)').alias('amount'))

stats_df.printSchema()
stats_df.show()

root
 |-- customer_id: long (nullable = true)
 |-- date: date (nullable = true)
 |-- amount: long (nullable = true)

+-----------+----------+------+
|customer_id|      date|amount|
+-----------+----------+------+
|         75|1953-03-24|   159|
|         83|1940-01-21|    75|
|         57|1901-05-24|   287|
|         13|1955-07-13|   816|
|         17|1972-02-02|   292|
|         34|1992-04-27|   611|
|          2|1928-01-10|   427|
|         52|1909-07-14|   872|
|         67|1921-12-03|   309|
|         34|1946-06-01|   586|
|         86|1909-02-12|   874|
|         50|1982-03-29|   210|
|         71|2012-01-20|   611|
|         40|1957-07-20|   347|
|          4|1992-03-23|   473|
|         77|1999-09-08|   178|
|         29|1901-04-13|   562|
|         13|2015-06-12|    79|
|         14|2002-11-16|   225|
|         36|1975-09-13|   853|
+-----------+----------+------+
only showing top 20 rows



Spark permite cargar información de múltiples fuentes. A continuación se muestra como cargar datos de un archivo JSON que contiene datos del NYT Best Sellers.

## Ejemplo 2 NYT Best Sellers.

A partir de un archivo json se creará un spark dataframe y se transformará los datos. 

In [15]:
# Crea Dataframes de Spark a partir de  datos en formato JSON

dataframe = spark.read.json('nyt2.json')

#Archivos TXT# 
#dataframe = spark.read.text('text_data.txt')

#Archivos CSV
#dataframe_csv = spark.read.csv('csv_data.csv')

#Archivos PARQUET
# El formato Parquet es un formato open-source 
# utilizado para la serialización de datos (Apache Avro).
# El formato admite una compresión y codificación muy 
# eficientes de datos orientados a columnas.
#dataframe_parquet = spark.read.load('parquet_data.parquet')

## Ejemplos de uso

### 1. Exploración del conjunto de datos

In [16]:
print("DATOS GENERALES DEL DATAFRAME")
print("Cantidad de registros", dataframe.count(), "\n")

print("Desplegar las columnas", dataframe.describe(), "\n")

print("Desplegar los datos de algunas columnas")
dataframe.select("author", "title").show(10)

print("Desplegar todos los datos") 
dataframe.show(10)


DATOS GENERALES DEL DATAFRAME
Cantidad de registros 10195 

Desplegar las columnas DataFrame[summary: string, amazon_product_url: string, author: string, description: string, publisher: string, title: string] 

Desplegar los datos de algunas columnas
+--------------------+--------------------+
|              author|               title|
+--------------------+--------------------+
|       Dean R Koontz|           ODD HOURS|
|     Stephenie Meyer|            THE HOST|
|        Emily Giffin|LOVE THE ONE YOU'...|
|   Patricia Cornwell|           THE FRONT|
|     Chuck Palahniuk|               SNUFF|
|James Patterson a...|SUNDAYS AT TIFFANY’S|
|       John Sandford|        PHANTOM PREY|
|       Jimmy Buffett|          SWINE NOT?|
|    Elizabeth George|     CARELESS IN RED|
|      David Baldacci|     THE WHOLE TRUTH|
+--------------------+--------------------+
only showing top 10 rows

Desplegar todos los datos
+--------------------+--------------------+--------------------+-----------------

### 2. Remover duplicados

In [17]:
# El archivo no tiene registros duplicados por lo que primero 
# se carga dos veces los datos para generar duplicados
dataframe2 = spark.read.json('nyt2.json')

# Concatenar dos dataframes 
df_concat = dataframe.union(dataframe2)


print("Cantidad de registros antes", df_concat.count(), "\n")

dataframe_dropdup = df_concat.dropDuplicates() 
print("Cantidad de registros después de borrar duplicados", dataframe_dropdup.count(), "\n")


Cantidad de registros antes 20390 

Cantidad de registros después de borrar duplicados 10195 



### Reemplazo when

In [18]:
dataframe.select("author", "title", "rank", "price").show(10)

dataframe.select("title",when(dataframe.title != 'ODD HOURS',1).otherwise(0).alias("result")).show(10)

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|       Dean R Koontz|           ODD HOURS| [1]|  [, 27]|
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       John Sandford|        PHANTOM PREY| [7]|[26.95,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth George|     CARELESS IN RED| [9]|[27.95,]|
|      David Baldacci|     THE WHOLE TRUTH|[10]|[26.99,]|
+--------------------+--------------------+----+--------+
only showing top 10 rows

+--------------------+------+
|               title|result|
+--------------------+------+
|           ODD HOURS|     0|
|            THE HOST|     1|
|LOVE THE ON

### Condicionales: isin y is not in

In [19]:
# is not (~) in
df_filtered = dataframe.filter(~dataframe["title"].isin("ODD HOURS", "PHANTOM PREY"))

print(df_filtered.select("author", "title", "rank", "price").show(10))

# Is in 
df_filtered = dataframe.filter(dataframe["title"].isin("ODD HOURS", "PHANTOM PREY"))

print(df_filtered.select("author", "title", "rank", "price").show(2))

+--------------------+--------------------+----+--------+
|              author|               title|rank|   price|
+--------------------+--------------------+----+--------+
|     Stephenie Meyer|            THE HOST| [2]|[25.99,]|
|        Emily Giffin|LOVE THE ONE YOU'...| [3]|[24.95,]|
|   Patricia Cornwell|           THE FRONT| [4]|[22.95,]|
|     Chuck Palahniuk|               SNUFF| [5]|[24.95,]|
|James Patterson a...|SUNDAYS AT TIFFANY’S| [6]|[24.99,]|
|       Jimmy Buffett|          SWINE NOT?| [8]|[21.99,]|
|    Elizabeth George|     CARELESS IN RED| [9]|[27.95,]|
|      David Baldacci|     THE WHOLE TRUTH|[10]|[26.99,]|
|        Troy Denning|          INVINCIBLE|[11]|  [, 27]|
|          James Frey|BRIGHT SHINY MORNING|[12]|[26.95,]|
+--------------------+--------------------+----+--------+
only showing top 10 rows

None
+-------------+------------+----+--------+
|       author|       title|rank|   price|
+-------------+------------+----+--------+
|Dean R Koontz|   ODD HOURS|

In [20]:
# Like
dataframe.select("title", dataframe.title.like("%THE%")).show()


+--------------------+----------------+
|               title|title LIKE %THE%|
+--------------------+----------------+
|           ODD HOURS|           false|
|            THE HOST|            true|
|LOVE THE ONE YOU'...|            true|
|           THE FRONT|            true|
|               SNUFF|           false|
|SUNDAYS AT TIFFANY’S|           false|
|        PHANTOM PREY|           false|
|          SWINE NOT?|           false|
|     CARELESS IN RED|           false|
|     THE WHOLE TRUTH|            true|
|          INVINCIBLE|           false|
|BRIGHT SHINY MORNING|           false|
|THE ART OF RACING...|            true|
|       TWENTY WISHES|           false|
|      THE STEEL WAVE|            true|
| EXECUTIVE PRIVILEGE|           false|
|  UNACCUSTOMED EARTH|           false|
|          NETHERLAND|            true|
|          THE APPEAL|            true|
|INDIANA JONES AND...|            true|
+--------------------+----------------+
only showing top 20 rows



In [21]:
# startswith y endswith 

dataframe.select("author", "title", dataframe.title.startswith("THE").alias("with_the")).show(5)



+-----------------+--------------------+--------+
|           author|               title|with_the|
+-----------------+--------------------+--------+
|    Dean R Koontz|           ODD HOURS|   false|
|  Stephenie Meyer|            THE HOST|    true|
|     Emily Giffin|LOVE THE ONE YOU'...|   false|
|Patricia Cornwell|           THE FRONT|    true|
|  Chuck Palahniuk|               SNUFF|   false|
+-----------------+--------------------+--------+
only showing top 5 rows



In [21]:
dataframe.select("author", "title", dataframe.title.endswith("NT").alias("ends_with_nt")).show(15)

+--------------------+--------------------+------------+
|              author|               title|ends_with_nt|
+--------------------+--------------------+------------+
|       Dean R Koontz|           ODD HOURS|       false|
|     Stephenie Meyer|            THE HOST|       false|
|        Emily Giffin|LOVE THE ONE YOU'...|       false|
|   Patricia Cornwell|           THE FRONT|        true|
|     Chuck Palahniuk|               SNUFF|       false|
|James Patterson a...|SUNDAYS AT TIFFANY’S|       false|
|       John Sandford|        PHANTOM PREY|       false|
|       Jimmy Buffett|          SWINE NOT?|       false|
|    Elizabeth George|     CARELESS IN RED|       false|
|      David Baldacci|     THE WHOLE TRUTH|       false|
|        Troy Denning|          INVINCIBLE|       false|
|          James Frey|BRIGHT SHINY MORNING|       false|
|         Garth Stein|THE ART OF RACING...|       false|
|     Debbie Macomber|       TWENTY WISHES|       false|
|         Jeff Shaara|      THE

In [22]:
# Substring

dataframe.select(dataframe.author.substr(1, 6).alias("Substr")).show(5)

+------+
|Substr|
+------+
|Dean R|
|Stephe|
|Emily |
|Patric|
|Chuck |
+------+
only showing top 5 rows



In [26]:
# Consultas SQL

#Total de registros
print("Total de registros", dataframe2.count())

# Se debe primero registrar la tabla
dataframe.registerTempTable("books")

spark.sql("SELECT title, author FROM books").show()

spark.sql("SELECT * FROM books").show()

spark.sql("SELECT * FROM books where title = 'ODD HOURS'").show(1)


Total de registros 10195
+--------------------+--------------------+
|               title|              author|
+--------------------+--------------------+
|           ODD HOURS|       Dean R Koontz|
|            THE HOST|     Stephenie Meyer|
|LOVE THE ONE YOU'...|        Emily Giffin|
|           THE FRONT|   Patricia Cornwell|
|               SNUFF|     Chuck Palahniuk|
|SUNDAYS AT TIFFANY’S|James Patterson a...|
|        PHANTOM PREY|       John Sandford|
|          SWINE NOT?|       Jimmy Buffett|
|     CARELESS IN RED|    Elizabeth George|
|     THE WHOLE TRUTH|      David Baldacci|
|          INVINCIBLE|        Troy Denning|
|BRIGHT SHINY MORNING|          James Frey|
|THE ART OF RACING...|         Garth Stein|
|       TWENTY WISHES|     Debbie Macomber|
|      THE STEEL WAVE|         Jeff Shaara|
| EXECUTIVE PRIVILEGE|    Phillip Margolin|
|  UNACCUSTOMED EARTH|       Jhumpa Lahiri|
|          NETHERLAND|      Joseph O'Neill|
|          THE APPEAL|        John Grisham|
|INDIAN

In [27]:
### Agregando, borrando y renombrando columnas

# Agregar columna
dataframe2 = dataframe.withColumn('new_column', dataframe.title)
display(dataframe2)

print(dataframe2.select('new_column',"title", "amazon_product_url").show(10))



DataFrame[_id: struct<$oid:string>, amazon_product_url: string, author: string, bestsellers_date: struct<$date:struct<$numberLong:string>>, description: string, price: struct<$numberDouble:string,$numberInt:string>, published_date: struct<$date:struct<$numberLong:string>>, publisher: string, rank: struct<$numberInt:string>, rank_last_week: struct<$numberInt:string>, title: string, weeks_on_list: struct<$numberInt:string>, new_column: string]

+--------------------+--------------------+--------------------+
|          new_column|               title|  amazon_product_url|
+--------------------+--------------------+--------------------+
|           ODD HOURS|           ODD HOURS|http://www.amazon...|
|            THE HOST|            THE HOST|http://www.amazon...|
|LOVE THE ONE YOU'...|LOVE THE ONE YOU'...|http://www.amazon...|
|           THE FRONT|           THE FRONT|http://www.amazon...|
|               SNUFF|               SNUFF|http://www.amazon...|
|SUNDAYS AT TIFFANY’S|SUNDAYS AT TIFFANY’S|http://www.amazon...|
|        PHANTOM PREY|        PHANTOM PREY|http://www.amazon...|
|          SWINE NOT?|          SWINE NOT?|http://www.amazon...|
|     CARELESS IN RED|     CARELESS IN RED|http://www.amazon...|
|     THE WHOLE TRUTH|     THE WHOLE TRUTH|http://www.amazon...|
+--------------------+--------------------+--------------------+
only showing top 10 rows

None


In [28]:
# Renombrar columna
print("Renombrar columna")
dataframe3 = dataframe2.withColumnRenamed('amazon_product_url', 'URL')
dataframe3.show(5)



Renombrar columna
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|                 URL|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|http://www.amazon...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|           ODD HOURS|
|[5b4aa4ead3089013...|http://www.amazon...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[

In [29]:
# Borrar una columna
dataframe_remove = dataframe3.drop("URL").show(5)

+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|                 _id|           author| bestsellers_date|         description|   price|   published_date|    publisher|rank|rank_last_week|               title|weeks_on_list|          new_column|
+--------------------+-----------------+-----------------+--------------------+--------+-----------------+-------------+----+--------------+--------------------+-------------+--------------------+
|[5b4aa4ead3089013...|    Dean R Koontz|[[1211587200000]]|Odd Thomas, who c...|  [, 27]|[[1212883200000]]|       Bantam| [1]|           [0]|           ODD HOURS|          [1]|           ODD HOURS|
|[5b4aa4ead3089013...|  Stephenie Meyer|[[1211587200000]]|Aliens have taken...|[25.99,]|[[1212883200000]]|Little, Brown| [2]|           [1]|            THE HOST|          [3]|            THE HOST|
|[5b4aa4ead3089

In [31]:
# Salvar datos a disco en un formato particular
dataframe.select("author", "title") \
.write \
.save("Authors_Titles2.json",format="json")



In [32]:
dataframe.select("author", "title") \
.write \
.save("Authors_Titles3.csv",format="csv")

In [34]:


#Salvar datos a PostgreSQL
def write_spark_df_to_db(spark_df, table_name):
    """
    This function writes Spark dataframe to DB
    """
    spark_df \
        .write \
        .format("jdbc") \
        .mode('overwrite') \
        .option("url", POSTGRESQL_URL) \
        .option("user", POSTGRESQL_USER) \
        .option("password", POSTGRESQL_PASSWORD) \
        .option("dbtable", table_name) \
        .save()
    
write_spark_df_to_db(dataframe.select("author", "title"), "prueba")    

# Referencias

[1] Spark SQL, DataFrames and Datasets Guide. Recuperado de https://spark.apache.org/docs/2.3.0/sql-programming-guide.html

[2] Esquivel, J. (2019). Big Data. Ciencias de los datos. ITCR.